## Capstone

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
# Grab the data from here: http://share.mailcharts.com/0D0Q2e0L1s47 and http://share.mailcharts.com/1X3k1c1H0v3M
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv("./data/capstone-v2.csv", encoding="ISO-8859-1")
print(df.shape)
print(df.columns)

(3365, 8)
Index(['reg_id', 'add_id', 'name', 'email_guid', 'sent_at', 'subject',
       'full_text', 'r'],
      dtype='object')


In [44]:
df["all_text"] = df["subject"] + " " + df["full_text"].fillna("")

In [200]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import ngrams
import string
import re

# Steps: Clean up text, stemming, remove stop words and weird chars, tokenizer words

# punctuation = list(set(string.punctuation))
re_punctuation = "\#|\.|\>|\/|\)|\"|\(|\}|\'|\_|\-|\$|\:|\[|\^|\+|\?|\`|\~|\!|\<|\@|\;|\=|\*|\\\|\{|\&|\]|\||\,|\|"
stopwords_set = list(set(stopwords.words('english')))
handpicked_works = ["com"]

def get_unigram_sentence(sentence, company_name):
    company_names = company_name.lower().split(" ")
    company_names.append(company_name.lower().replace(" ", ""))
    
    sentence_no_punc = re.sub(re_punctuation, " ", sentence)
    unigram = [word for word in word_tokenize(sentence_no_punc.lower()) if word not in stopwords_set and word not in company_names and word.isalpha() and word not in handpicked_works]
    return unigram

In [201]:
tokenized_text = []

# We're only looking at subject lines!
for i, el in df.iterrows():
    tokenized_text.append(get_unigram_sentence(el['subject'], el['name']))

df["tokenized_text"] = tokenized_text

In [202]:
stemmer = PorterStemmer()

def get_stems(words):
    return [stemmer.stem(word) for word in words]

In [203]:
df["stemmed_tokens"] = df.tokenized_text.apply(lambda x: get_stems(x))

In [204]:
df["stemmed_text"] = df["stemmed_tokens"].apply(lambda x: " ".join(word for word in x))

In [205]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english", min_df=50)
tfidf = tfidf.fit(df["stemmed_text"])
X = tfidf.transform(df["stemmed_text"])
X = X.toarray()

//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [206]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size=0.25, random_state=100)
df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

print(X_train.shape)
print(X_test.shape)

(2523, 26)
(842, 26)


In [207]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(6, random_state = 100)
lda.fit(X_train)

//anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=100,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [233]:
labels = []

def print_topics(model, vectorizer, top_n=5):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

In [234]:
print_topics(lda, tfidf)

Topic 0:
[('order', 276.5700312415117), ('confirm', 134.5845927648888), ('thank', 54.10736202184687), ('deal', 49.5085950336204), ('shop', 39.103718794612774)]
Topic 1:
[('new', 129.70842762796025), ('save', 75.95254190973414), ('way', 56.077224726495814), ('shop', 9.202492706908524), ('sale', 0.16777982670933952)]
Topic 2:
[('sale', 79.17263543236022), ('today', 55.045833394647275), ('purchas', 48.477522886799285), ('end', 48.33724792657309), ('shipment', 39.07170970795043)]
Topic 3:
[('ship', 160.87928044934705), ('day', 67.75448022606383), ('order', 66.18839824401725), ('lorem', 59.988955351807654), ('free', 49.760784001061936)]
Topic 4:
[('account', 58.88465786215551), ('style', 44.53486553855523), ('holiday', 38.31875898345485), ('deliveri', 34.48444336257383), ('free', 34.296012702618164)]
Topic 5:
[('welcom', 197.05497511304637), ('gift', 61.10171688714177), ('thank', 49.10061464895268), ('receiv', 36.353400650753386), ('order', 15.619523937040583)]


In [210]:
# Next, let's assign this to our original text

X_train_tfidf_lda = lda.transform(X_train)

predicted_label = []

for i in X_train_tfidf_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df_train["predicted_label"] = predicted_label

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [211]:
df_train.head(10)

,reg_id,add_id,name,email_guid,sent_at,subject,full_text,r,all_text,tokenized_text,stemmed_tokens,stemmed_text,predicted_label
141,7916,9086,Leslie's Poolmart,d3d1d897-bad3-d5cf-51c7-2f3359d5c62e,2017-02-02 15:19:06,Thank you for your purchase. Order number: A72...,Having trouble reading this email? | View this...,1,Thank you for your purchase. Order number: A72...,"[thank, purchase, order, number]","[thank, purchas, order, number]",thank purchas order number,order
2539,39085,39393,Express,4d040549-f45c-291c-1bc4-63a33edede04,2017-12-13 01:02:22,Your order has shipped!,Thanks for shopping with us! We have new style...,2,Your order has shipped! Thanks for shopping wi...,"[order, shipped]","[order, ship]",order ship,ship
1421,31065,32312,DICK'S Sporting Goods,5d1cecff-a92f-cd65-0df1-1c1072b615ec,2017-10-14 00:26:00,PSSTâ¦.Something BIG Starts Tomorrowâ¦,2 Days Only! 20% Off Your Entire Purchase** We...,5,PSSTâ¦.Something BIG Starts Tomorrowâ¦ 2 Day...,"[something, big, starts]","[someth, big, start]",someth big start,order
2950,39398,39707,Free People,35a5528a-cc1d-301c-7c79-d4010a105e73,2017-12-22 13:47:52,Your Order Confirmation,Free People Order Confirmation Order Number: F...,1,Your Order Confirmation Free People Order Conf...,"[order, confirmation]","[order, confirm]",order confirm,order
1265,30856,32103,Chico's,d4a5828a-0dcc-d992-6c87-c7dd6cd11eca,2017-10-05 11:18:49,Happy Early Shopping (40% Off Everything),Friends donât let friends miss a deal like t...,9,Happy Early Shopping (40% Off Everything) Frie...,"[happy, early, shopping, everything]","[happi, earli, shop, everyth]",happi earli shop everyth,order
1995,37987,38286,ALDO,39875bc0-3a6c-6a7d-b206-fbaea3c93678,2017-11-23 14:24:35,50% Off. Black Friday on Now. Let's do this + ...,"Itâs cool, weâre just as excited as you! I...",8,50% Off. Black Friday on Now. Let's do this + ...,"[black, friday, let, free, shipping]","[black, friday, let, free, ship]",black friday let free ship,ship
1493,31083,32330,Hollister Co.,3698172c-0bed-f436-b67b-4e093012a934,2017-10-11 11:46:03,TODAY ONLY: All jeans $25!,"Plus, free shipping with any jeans purchase. V...",9,"TODAY ONLY: All jeans $25! Plus, free shipping...","[today, jeans]","[today, jean]",today jean,sale
3147,39435,39744,Target,7b91c7ef-5052-7140-f4a1-acf374562f8e,2018-01-01 17:33:58,Last days to save big on storage.,Making your inbox happy. âÂ âÂ âÂ âÂ â...,9,Last days to save big on storage. Making your ...,"[last, days, save, big, storage]","[last, day, save, big, storag]",last day save big storag,ship
1411,31062,32309,CVS Pharmacy,4a57d0b1-e8ed-b4a0-6daa-061494f00465,2017-10-30 14:17:01,Email Exclusive: 31% Off Now!,2 days only. Gobblin up these savings on spook...,5,Email Exclusive: 31% Off Now! 2 days only. Gob...,"[email, exclusive]","[email, exclus]",email exclus,welcom
1920,37887,38186,Duluth Trading Company,ff0f8b1e-d6cf-f527-3f70-c612c0a32df0,2017-11-15 12:21:14,$14.50 Men's Buck Naked Underwear - Whack And ...,Plus chop your gift list down to size with $16...,3,$14.50 Men's Buck Naked Underwear - Whack And ...,"[men, buck, naked, underwear, whack, hack, wee...","[men, buck, nake, underwear, whack, hack, week...",men buck nake underwear whack hack week special,welcom


In [243]:
# we want to look at "ship" and "order"
# we want to re-classify them using full-text and subject line

retrain_df = df_train[(df_train["predicted_label"] == "ship") | (df_train["predicted_label"] == "order")]

In [244]:
# TFIDF

tfidf_retrain = TfidfVectorizer(stop_words="english", min_df=50)
tfidf_retrain = tfidf_retrain.fit(retrain_df["all_text"])
X_retrain = tfidf_retrain.transform(retrain_df["all_text"])
X_retrain = X_retrain.toarray()

//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [269]:
# LDA

lda_retrain = LDA(2, random_state = 100)
lda_retrain.fit(X_retrain)

//anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=2, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=100,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [270]:
retrain_labels = []

def print_topics(model, vectorizer, top_n=5):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        retrain_labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

In [271]:
print_topics(lda_retrain, tfidf_retrain)

Topic 0:
[('order', 119.63635102862926), ('shipped', 41.23067452100446), ('shipping', 41.0764144007272), ('00', 37.638265505945036), ('lorem', 37.46430246848148)]
Topic 1:
[('shop', 48.430524836540215), ('free', 45.290343029765644), ('offer', 34.4676092048357), ('shipping', 32.27586556941814), ('valid', 30.547721669584895)]


In [272]:
X_retrain_tfidf_lda = lda_retrain.transform(X_retrain)

predicted_label_retrain = []

for i in X_retrain_tfidf_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label_retrain.append(retrain_labels[i.argsort()[-1]])

retrain_df["predicted_label_retrain"] = predicted_label_retrain

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [273]:
retrain_df[["subject", "predicted_label", "predicted_label_retrain"]]

# X, order => x
# X, shop => sales

,subject,predicted_label,predicted_label_retrain
141,Thank you for your purchase. Order number: A72...,order,order
2539,Your order has shipped!,ship,order
1421,PSSTâ¦.Something BIG Starts Tomorrowâ¦,order,shop
2950,Your Order Confirmation,order,order
1265,Happy Early Shopping (40% Off Everything),order,shop
1995,50% Off. Black Friday on Now. Let's do this + ...,ship,shop
3147,Last days to save big on storage.,ship,shop
902,Thanks for your order from Bleecker's Finest Deli,order,order
1292,Newsletter Signup: Subscription Confirmed,order,shop
343,â Dad Approved Fathers Day Ideas + A Coupon â,ship,shop
